<b><u>Observations</b></u>
<p>1. observation</p>
<p>2. observation</p>
<p>3. observation</p>

In [1]:
# Dependencies
import random
import json
import requests
from pprint import pprint
from config import api_key
import pandas as pd
import seaborn as sns
from citipy import citipy
import matplotlib.pyplot as plt

# Set the seaborn chart style
sns.set()

# Dependencies for downloading and uncompressing list of cities from OpenWeatherMap
import urllib.request
import gzip

<h1>Download City List</h1>

In [3]:
# This section downloads the list of cities and IDs that OpenWeatherMap uses

# File names
city_index_zip_file = "city.list.json.gz"
city_index_json_file = 'city.list.json'

# Download file
urllib.request.urlretrieve ("http://bulk.openweathermap.org/sample/city.list.json.gz", city_index_zip_file)

# Uncompress gzip file to json
inF = gzip.open(city_index_zip_file, 'rb')
outF = open(city_index_json_file, 'wb')
outF.write( inF.read() )
inF.close()
outF.close()

# Read json file into a dataframe
city_index_df = pd.read_json(city_index_json_file)

# Print top five rows
city_index_df.head()

,coord,country,id,name
0,"{'lon': 34.283333, 'lat': 44.549999}",UA,707860,Hurzuf
1,"{'lon': 37.666668, 'lat': 55.683334}",RU,519188,Novinki
2,"{'lon': 84.633331, 'lat': 28}",NP,1283378,GorkhÄ�
3,"{'lon': 76, 'lat': 29}",IN,1270260,State of HaryÄ�na
4,"{'lon': 33.900002, 'lat': 44.599998}",UA,708546,Holubynka


<h1>Generate City List</h1>

In [14]:
# This section selects 500 random cities from the city index list

# Start by creating an empty dataframe
random_city_df = pd.DataFrame(columns=['coord', 'country', 'id', 'name', 'lon', 'lat'])

# Set counter to zero for looping
counter = 0

# Set total number of random cities
total_cities = 5

# Loop while counter is less than number of cities
while counter < total_cities:
    
    # Pick a random row (must be length - 1 since row starts at 0)
    row = random.randint(0, len(city_index_df)-1)
    
    # If ID is already in the list then do nothing repeat loop
    if any(random_city_df.id == city_index_df['id'][row]):
        pass
    
    # If ID isn't in the list then add that row to the dataframe, including latitude and longitude
    else:
        random_city_df.loc[counter] = city_index_df.iloc[row,:]
        random_city_df.lat[counter] = city_index_df['coord'][row]['lat']
        random_city_df.lon[counter] = city_index_df['coord'][row]['lon']
        counter = counter + 1
        
# Change latitude and longitude columns from object to float
random_city_df[['lat','lon']] = random_city_df[['lat','lon']].apply(pd.to_numeric)

# Remove coordinate column, since we parsed out that data
del random_city_df['coord']

print(random_city_df.id)

0    2843673
1    6456283
2     127082
3    2889674
4    6615440
Name: id, dtype: object


<h1>Weather API Call</h1>

In [15]:
# Let's get the JSON for 100 posts sequentially.
url = "http://api.openweathermap.org/data/2.5/weather?APPID="+api_key+"&id="+str(random_city_df.id[0])
url

'http://api.openweathermap.org/data/2.5/weather?APPID=f794c4f97402e541fc87718afc020f36&id=2843673'

In [17]:
response = requests.get(url)
data = response.json()
pprint(data)

'''
# Make a request for each city
for x in range(total_cities):
    print(f"Request {x+1} of {total_cities}: {city_list[x]}")
    print('url')

    # Get one of the posts
    post_response = requests.get(url)

    # Save post's JSON
    response_json.append(post_response.json())
'''

{'base': 'stations',
 'clouds': {'all': 1},
 'cod': 200,
 'coord': {'lat': 49.51, 'lon': 9.97},
 'dt': 1521860160,
 'id': 2843673,
 'main': {'humidity': 88,
          'pressure': 1003,
          'temp': 273.15,
          'temp_max': 273.15,
          'temp_min': 273.15},
 'name': 'Rottingen',
 'sys': {'country': 'DE',
         'id': 4966,
         'message': 0.0043,
         'sunrise': 1521868469,
         'sunset': 1521913149,
         'type': 1},
 'visibility': 10000,
 'weather': [{'description': 'mist', 'icon': '50n', 'id': 701, 'main': 'Mist'}],
 'wind': {'deg': 70, 'speed': 1.5}}


'\n# Make a request for each city\nfor x in range(total_cities):\n    print(f"Request {x+1} of {total_cities}: {city_list[x]}")\n    print(\'url\')\n\n    # Get one of the posts\n    post_response = requests.get(url)\n\n    # Save post\'s JSON\n    response_json.append(post_response.json())\n'

<h1>Print Charts</h1>

In [ ]:
# This section prints a scatter plot of latitude by temperature

# Create bubble plots
plt.scatter(latitude_list, max_temp, marker="o", facecolors='gold', edgecolors="black", alpha=1)

# Create a title, x label, and y label
plt.title("City Temperature by Temperature (F\N{DEGREE SIGN})", fontsize=20, fontweight="bold")
plt.xlabel("Latitude", fontsize=14)
plt.ylabel("Temperature (F\N{DEGREE SIGN})", fontsize=14)

#plt.colorbar()

# Set size of x and y tick labels
plt.tick_params(labelsize=12)

# Set x and y limits
#plt.xlim(0, max(rural_total_rides_by_city.max(), urban_total_rides_by_city.max(), suburban_total_rides_by_city.max())+5)
#plt.ylim(15, max(rural_avg_fare_by_city.max(), urban_avg_fare_by_city.max(), suburban_avg_fare_by_city.max())+5)

# Display bubble chart
plt.show()

<h1>Export Results</h1>